# Baseline - klasyfikacja



In [171]:
import srai
import pandas as pd
import geopandas as gpd
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.embedders import CountEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMOnlineLoader
from srai.plotting import plot_regions
import overpy
import time
import pickle
from srai.constants import WGS84_CRS, FEATURES_INDEX

## Tworzenie zbioru danych

Wykorzystane tagi: highway, amenity, building

Dane wypadków (ilość) na rok 2023

In [17]:
area = geocode_to_region_gdf("Wrocław, Poland")

In [31]:
area.columns

Index(['geometry'], dtype='object')

In [117]:
def get_all_values_for_tag(tag):
    """
    Gets all unique values for given OSM key
    """
    api = overpy.Overpass()
    query = f"""
    area[name="Wrocław"];
    (nwr[{tag}](area););
    out tags;
    """ 
    values_all = []
# Execute the query
    result = api.query(query)
    for way in result.ways:
        for key, value in way.tags.items():
            if key ==tag:
                print(f"Way {way.id}: {key} = {value}")
                values_all.append(value)

    for node in result.nodes:
        for key, value in node.tags.items():
            if key ==tag:
                print(f"Node {node.id}: {key} = {value}")
                values_all.append(value)

    for relation in result.relations:
        for key, value in relation.tags.items():
            if key ==tag:
                print(f"Relation {relation.id}: {key} = {value}")
                values_all.append(value)
    values_set = set(values_all)
    return list(values_set)

In [70]:
def slice_list(input_list, max_elements_per_slice=20):
    sliced_list = []
    for i in range(0, len(input_list), max_elements_per_slice):
        sliced_list.append(input_list[i:i + max_elements_per_slice])
    return sliced_list

In [132]:
def get_list_of_features_gdf(tag_key_names):
    features_dict = {}
    for tag_key in tag_key_names:
        tag_values_list = get_all_values_for_tag(tag_key)
        sliced_values = slice_list(tag_values_list, max_elements_per_slice=10)
        features_dict[tag_key] = sliced_values
  
    features_list = []
    for key, val in features_dict.items():
        for i, inner in enumerate(val):
            query = {key: inner}
            loader = OSMOnlineLoader()
            features = loader.load(area, query)
            features_list.append(features)
            time.sleep(30)
    
    return features_list
        


In [163]:
download_features = False
if download_features:
    feature_key_names = ["highway", "amenity"]
    features_list1 = get_list_of_features_gdf(feature_key_names)
    feature_key_names2 = ["building"]
    features_list2 = get_list_of_features_gdf(feature_key_names2)
    features_list1.extend(features_list2)
    concat_gdf = gpd.GeoDataFrame(pd.concat(features_list1, ignore_index=False), crs=features_list1[0].crs)
    concat_gdf.to_pickle("data/features_gdf_highway_amenity_building.pkl")

In [139]:
all_features_gdf = pd.read_pickle("data/features_gdf_highway_amenity_building.pkl")

In [142]:
all_features_gdf.head()

,geometry,highway,amenity,building
feature_id,,,,
node/150597406,POINT (16.97786 51.09238),crossing,NaN,NaN
node/151334674,POINT (16.97939 51.09485),traffic_signals,NaN,NaN
node/153568043,POINT (16.96694 51.07828),crossing,NaN,NaN
node/158719856,POINT (17.03100 51.09441),crossing,NaN,NaN
node/158719876,POINT (17.04118 51.09612),traffic_signals,NaN,NaN


In [153]:
duplicates = all_features_gdf.index.duplicated(keep='first')
features_no_duplicates = all_features_gdf[~duplicates]

In [182]:
wypadki = gpd.read_file("./data/wypadki/2023_szczegoly_zdarzen_dr.shx")
wypadki = wypadki.to_crs('EPSG:4326')

In [183]:
def make_count_embeddings(def_area, features, regionalizer):
    # the indexes should be unique for count embedder
    assert len(features.index.unique()) == len(features)
    
    regions = regionalizer.transform(def_area)
    joiner = IntersectionJoiner()
    joint = joiner.transform(regions, features)
    embedder = CountEmbedder()
    embeddings = embedder.transform(regions, features, joint)
    return embeddings

def make_count_embeddings_wypadki(def_area, wypadki_df, regionalizer):
    query = {'rok': wypadki_df['ROK'], 'geometry': wypadki_df['geometry']}
    features_y = gpd.GeoDataFrame(
        query,
        crs=WGS84_CRS,
        index=gpd.pd.Index(name=FEATURES_INDEX, data=[i for i in range(len(wypadki_df))]),
    )
    regions = regionalizer.transform(def_area)
    joiner = IntersectionJoiner()
    joint = joiner.transform(regions, features_y, return_geom=True)
    embedder = CountEmbedder()
    embeddings = embedder.transform(regions, features_y, joint)
    return embeddings
   

In [184]:
regionalizer = H3Regionalizer(resolution=9)
embeddings_features = make_count_embeddings(area, features_no_duplicates, regionalizer)
embeddings_y = make_count_embeddings_wypadki(area, wypadki, regionalizer )


/home/gry-mar/osm-proj/gradient/env/lib/python3.8/site-packages/h3ronpy/arrow/vector.py:147: DeprecationWarning: The all_intersecting parameter is deprecated and will be removed. Use containment_mode instead.
  return vector.wkb_to_cells(
/home/gry-mar/osm-proj/gradient/env/lib/python3.8/site-packages/h3ronpy/arrow/vector.py:147: DeprecationWarning: The all_intersecting parameter is deprecated and will be removed. Use containment_mode instead.
  return vector.wkb_to_cells(


In [187]:
embeddings_y.index[0]

'891e204046bffff'

In [188]:
embeddings_features.index[0]

'891e204046bffff'

In [201]:
all_data = embeddings_features.join(embeddings_y)
all_data.head()

,highway_bridleway,highway_bus_stop,highway_construction,highway_corridor,highway_crossing,highway_cycleway,highway_elevator,highway_footway,highway_give_way,highway_living_street,highway_milestone,highway_mini_roundabout,highway_motorway,highway_motorway_junction,highway_motorway_link,highway_passing_place,highway_path,highway_pedestrian,highway_platform,highway_primary,highway_primary_link,highway_proposed,highway_raceway,highway_residential,highway_secondary,highway_secondary_link,highway_service,highway_steps,highway_stop,highway_street_lamp,highway_tertiary,highway_tertiary_link,highway_track,highway_traffic_mirror,highway_traffic_signals,highway_trunk,highway_trunk_link,highway_turning_circle,highway_turning_loop,highway_unclassified,amenity_animal_boarding,amenity_animal_shelter,amenity_animal_training,amenity_arts_centre,amenity_atm,amenity_baby_hatch,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,amenity_bicycle_parking,amenity_bicycle_rental,amenity_bicycle_repair_station,amenity_biergarten,amenity_boat_rental,amenity_bts,amenity_bureau_de_change,amenity_bus_station,amenity_cafe,amenity_car_rental,amenity_car_wash,amenity_casino,amenity_charging_station,amenity_childcare,amenity_cinema,amenity_clinic,amenity_clock,amenity_college,amenity_community_centre,amenity_compressed_air,amenity_conference_centre,amenity_courthouse,amenity_coworking_space,amenity_crematorium,amenity_dancing_school,amenity_dentist,amenity_doctors,amenity_dog_toilet,amenity_dog_toilets,amenity_dormitory,amenity_dressing_room,amenity_drinking_water,amenity_driver_training,amenity_driving_school,amenity_events_venue,amenity_fast_food,amenity_feeding_place,amenity_fire_station,amenity_fixme,amenity_food_court,amenity_food_sharing,amenity_fountain,amenity_fridge,amenity_fuel,amenity_give_box,amenity_grave_yard,amenity_grit_bin,amenity_guitar_lessons,amenity_hookah_lounge,amenity_hospital,amenity_hunting_stand,amenity_ice_cream,amenity_internet_cafe,amenity_kindergarten,amenity_language_school,amenity_letter_box,amenity_library,amenity_loading_dock,amenity_lounger,amenity_luggage_locker,amenity_marketplace,amenity_mist_spraying_cooler,amenity_monastery,amenity_money_transfer,amenity_motorcycle_parking,amenity_music_school,amenity_music_venue,amenity_nightclub,amenity_nursing_home,amenity_parcel_locker,amenity_parking,amenity_parking_entrance,amenity_parking_locker,amenity_parking_space,amenity_payment_centre,amenity_payment_terminal,amenity_pharmacy,amenity_photo_booth,amenity_place_of_worship,amenity_police,amenity_post_box,amenity_post_depot,amenity_post_office,amenity_printer,amenity_prison,amenity_pub,amenity_public_bath,amenity_public_bookcase,amenity_reception_desk,amenity_recycling,amenity_research_institute,amenity_restaurant,amenity_safe_deposit,amenity_school,amenity_shelter,amenity_shower,amenity_singing_lessons,amenity_smoking_area,amenity_social_centre,amenity_social_facility,amenity_stage,amenity_stripclub,amenity_studio,amenity_table,amenity_taxi,amenity_theatre,amenity_toilets,amenity_townhall,amenity_traffic_park,amenity_training,amenity_trolley_bay,amenity_university,amenity_vacuum_cleaner,amenity_vehicle_inspection,amenity_vending_machine,amenity_veterinary,amenity_waste_basket,amenity_waste_disposal,amenity_watering_place,amenity_weighbridge,building_allotment_house,building_apartments,building_barn,building_being_demolished,building_boathouse,building_bridge,building_bungalow,building_bunker,building_buttress,building_carport,building_castle,building_chapel,building_chimney,building_church,building_civic,building_collapsed,building_college,building_commercial,building_construction,building_container,building_convent,building_corridor,building_cover,building_data_center,building_detached,building_dormitory,building_elevator_housing,building_entrance,building_farm,building_farm_auxiliary,building_fire_station,building_fountain,building_garage,building_garages,building_gate,building_gazebo,building_government,building_grandstand,

## Czyszczenie danych
W pierwszej kolejności usunięto **amenity: waste_basket** jako tag, dla którego na pewno występuje niekonsekwencja tagowania ([ref](https://arxiv.org/pdf/2111.00970.pdf)). 
W kolejnym kroku usunięto rekordy, które nie posiadały żadnych cech (wszystkie features oprócz ilości wypadków == 0).

In [202]:
all_data = all_data.drop("amenity_waste_basket", axis=1)

In [203]:
all_data = all_data.rename(columns={'rok_2023': 'wypadki_count'})

In [204]:
all_data.isna().sum()

highway_bridleway          0
highway_bus_stop           0
highway_construction       0
highway_corridor           0
highway_crossing           0
                          ..
building_waste             0
building_water_tower       0
building_wayside_shrine    0
building_yes               0
wypadki_count              0
Length: 276, dtype: int64

In [205]:
all_data['wypadki_count'].describe()

count    2965.000000
mean        1.537943
std         3.986947
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        40.000000
Name: wypadki_count, dtype: float64

In [216]:
bins = [0, 10, 20, 30, 40]

# Use pd.cut to categorize the numeric column
categories = [0, 1, 2, 3]
all_data['wypadki_category'] = pd.cut(all_data['wypadki_count'], bins, labels=categories, include_lowest=True)


In [220]:
all_data.head()

,highway_bridleway,highway_bus_stop,highway_construction,highway_corridor,highway_crossing,highway_cycleway,highway_elevator,highway_footway,highway_give_way,highway_living_street,highway_milestone,highway_mini_roundabout,highway_motorway,highway_motorway_junction,highway_motorway_link,highway_passing_place,highway_path,highway_pedestrian,highway_platform,highway_primary,highway_primary_link,highway_proposed,highway_raceway,highway_residential,highway_secondary,highway_secondary_link,highway_service,highway_steps,highway_stop,highway_street_lamp,highway_tertiary,highway_tertiary_link,highway_track,highway_traffic_mirror,highway_traffic_signals,highway_trunk,highway_trunk_link,highway_turning_circle,highway_turning_loop,highway_unclassified,amenity_animal_boarding,amenity_animal_shelter,amenity_animal_training,amenity_arts_centre,amenity_atm,amenity_baby_hatch,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,amenity_bicycle_parking,amenity_bicycle_rental,amenity_bicycle_repair_station,amenity_biergarten,amenity_boat_rental,amenity_bts,amenity_bureau_de_change,amenity_bus_station,amenity_cafe,amenity_car_rental,amenity_car_wash,amenity_casino,amenity_charging_station,amenity_childcare,amenity_cinema,amenity_clinic,amenity_clock,amenity_college,amenity_community_centre,amenity_compressed_air,amenity_conference_centre,amenity_courthouse,amenity_coworking_space,amenity_crematorium,amenity_dancing_school,amenity_dentist,amenity_doctors,amenity_dog_toilet,amenity_dog_toilets,amenity_dormitory,amenity_dressing_room,amenity_drinking_water,amenity_driver_training,amenity_driving_school,amenity_events_venue,amenity_fast_food,amenity_feeding_place,amenity_fire_station,amenity_fixme,amenity_food_court,amenity_food_sharing,amenity_fountain,amenity_fridge,amenity_fuel,amenity_give_box,amenity_grave_yard,amenity_grit_bin,amenity_guitar_lessons,amenity_hookah_lounge,amenity_hospital,amenity_hunting_stand,amenity_ice_cream,amenity_internet_cafe,amenity_kindergarten,amenity_language_school,amenity_letter_box,amenity_library,amenity_loading_dock,amenity_lounger,amenity_luggage_locker,amenity_marketplace,amenity_mist_spraying_cooler,amenity_monastery,amenity_money_transfer,amenity_motorcycle_parking,amenity_music_school,amenity_music_venue,amenity_nightclub,amenity_nursing_home,amenity_parcel_locker,amenity_parking,amenity_parking_entrance,amenity_parking_locker,amenity_parking_space,amenity_payment_centre,amenity_payment_terminal,amenity_pharmacy,amenity_photo_booth,amenity_place_of_worship,amenity_police,amenity_post_box,amenity_post_depot,amenity_post_office,amenity_printer,amenity_prison,amenity_pub,amenity_public_bath,amenity_public_bookcase,amenity_reception_desk,amenity_recycling,amenity_research_institute,amenity_restaurant,amenity_safe_deposit,amenity_school,amenity_shelter,amenity_shower,amenity_singing_lessons,amenity_smoking_area,amenity_social_centre,amenity_social_facility,amenity_stage,amenity_stripclub,amenity_studio,amenity_table,amenity_taxi,amenity_theatre,amenity_toilets,amenity_townhall,amenity_traffic_park,amenity_training,amenity_trolley_bay,amenity_university,amenity_vacuum_cleaner,amenity_vehicle_inspection,amenity_vending_machine,amenity_veterinary,amenity_waste_disposal,amenity_watering_place,amenity_weighbridge,building_allotment_house,building_apartments,building_barn,building_being_demolished,building_boathouse,building_bridge,building_bungalow,building_bunker,building_buttress,building_carport,building_castle,building_chapel,building_chimney,building_church,building_civic,building_collapsed,building_college,building_commercial,building_construction,building_container,building_convent,building_corridor,building_cover,building_data_center,building_detached,building_dormitory,building_elevator_housing,building_entrance,building_farm,building_farm_auxiliary,building_fire_station,building_fountain,building_garage,building_garages,building_gate,building_gazebo,building_government,building_grandstand,building_greenhouse,b

In [219]:
all_data = all_data.drop(['wypadki_count'], axis=1)

In [221]:
x_columns = all_data.drop(['wypadki_category'], axis=1).columns
all_data_filtered = all_data[all_data[x_columns].sum(axis=1) != 0]
all_data_filtered.head()

,highway_bridleway,highway_bus_stop,highway_construction,highway_corridor,highway_crossing,highway_cycleway,highway_elevator,highway_footway,highway_give_way,highway_living_street,highway_milestone,highway_mini_roundabout,highway_motorway,highway_motorway_junction,highway_motorway_link,highway_passing_place,highway_path,highway_pedestrian,highway_platform,highway_primary,highway_primary_link,highway_proposed,highway_raceway,highway_residential,highway_secondary,highway_secondary_link,highway_service,highway_steps,highway_stop,highway_street_lamp,highway_tertiary,highway_tertiary_link,highway_track,highway_traffic_mirror,highway_traffic_signals,highway_trunk,highway_trunk_link,highway_turning_circle,highway_turning_loop,highway_unclassified,amenity_animal_boarding,amenity_animal_shelter,amenity_animal_training,amenity_arts_centre,amenity_atm,amenity_baby_hatch,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,amenity_bicycle_parking,amenity_bicycle_rental,amenity_bicycle_repair_station,amenity_biergarten,amenity_boat_rental,amenity_bts,amenity_bureau_de_change,amenity_bus_station,amenity_cafe,amenity_car_rental,amenity_car_wash,amenity_casino,amenity_charging_station,amenity_childcare,amenity_cinema,amenity_clinic,amenity_clock,amenity_college,amenity_community_centre,amenity_compressed_air,amenity_conference_centre,amenity_courthouse,amenity_coworking_space,amenity_crematorium,amenity_dancing_school,amenity_dentist,amenity_doctors,amenity_dog_toilet,amenity_dog_toilets,amenity_dormitory,amenity_dressing_room,amenity_drinking_water,amenity_driver_training,amenity_driving_school,amenity_events_venue,amenity_fast_food,amenity_feeding_place,amenity_fire_station,amenity_fixme,amenity_food_court,amenity_food_sharing,amenity_fountain,amenity_fridge,amenity_fuel,amenity_give_box,amenity_grave_yard,amenity_grit_bin,amenity_guitar_lessons,amenity_hookah_lounge,amenity_hospital,amenity_hunting_stand,amenity_ice_cream,amenity_internet_cafe,amenity_kindergarten,amenity_language_school,amenity_letter_box,amenity_library,amenity_loading_dock,amenity_lounger,amenity_luggage_locker,amenity_marketplace,amenity_mist_spraying_cooler,amenity_monastery,amenity_money_transfer,amenity_motorcycle_parking,amenity_music_school,amenity_music_venue,amenity_nightclub,amenity_nursing_home,amenity_parcel_locker,amenity_parking,amenity_parking_entrance,amenity_parking_locker,amenity_parking_space,amenity_payment_centre,amenity_payment_terminal,amenity_pharmacy,amenity_photo_booth,amenity_place_of_worship,amenity_police,amenity_post_box,amenity_post_depot,amenity_post_office,amenity_printer,amenity_prison,amenity_pub,amenity_public_bath,amenity_public_bookcase,amenity_reception_desk,amenity_recycling,amenity_research_institute,amenity_restaurant,amenity_safe_deposit,amenity_school,amenity_shelter,amenity_shower,amenity_singing_lessons,amenity_smoking_area,amenity_social_centre,amenity_social_facility,amenity_stage,amenity_stripclub,amenity_studio,amenity_table,amenity_taxi,amenity_theatre,amenity_toilets,amenity_townhall,amenity_traffic_park,amenity_training,amenity_trolley_bay,amenity_university,amenity_vacuum_cleaner,amenity_vehicle_inspection,amenity_vending_machine,amenity_veterinary,amenity_waste_disposal,amenity_watering_place,amenity_weighbridge,building_allotment_house,building_apartments,building_barn,building_being_demolished,building_boathouse,building_bridge,building_bungalow,building_bunker,building_buttress,building_carport,building_castle,building_chapel,building_chimney,building_church,building_civic,building_collapsed,building_college,building_commercial,building_construction,building_container,building_convent,building_corridor,building_cover,building_data_center,building_detached,building_dormitory,building_elevator_housing,building_entrance,building_farm,building_farm_auxiliary,building_fire_station,building_fountain,building_garage,building_garages,building_gate,building_gazebo,building_government,building_grandstand,building_greenhouse,b

In [222]:
len(all_data)

2965

In [223]:
len(all_data_filtered)

2883

## Przygotowanie modelu

In [224]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [225]:
X = all_data_filtered.drop(['wypadki_category'], axis=1)
y = all_data_filtered['wypadki_category']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)